In [27]:
import numpy as np
import pyvisa
import time

### Data collection

In [4]:
rm = pyvisa.ResourceManager()
rm.list_resources()

('ASRL4::INSTR', 'ASRL14::INSTR')

In [ ]:
with rm.open_resource('ASRL14::INSTR') as SPS30:
    time.sleep(2)
    try:
        while True:
            with open('SPS30.log', 'a') as f:
                resp = SPS30.query('r').strip()
                print(time.ctime(time.time()), ':', resp, end='\r')
                f.write(str(int(time.time())) + "," + resp + '\n')
                time.sleep(1)
    except KeyboardInterrupt:
        pass